In [5]:
# 构建训练集特征
# 预处理1： 栅格化图层
# 预处理2： 训练集对测试集采样
# 构造特征：基本特征、时间特征、空间特征
import pandas as pd
import numpy as np
import random
import time
import warnings
warnings.filterwarnings('ignore')

train_order = pd.read_csv('../../dataset/order_data/order_info_v2.csv')
test_order = pd.read_csv('../../dataset/order_data/test_order_cluster.csv')
# -------------------------- 数据读取 ---------------------------------
def get_train():
    COLUMNS = ['loadingOrder', 'carrierName', 'timestamp', 'longitude',
               'latitude', 'vesselMMSI', 'speed', 'direction', 'vesselNextport',
               'vesselNextportETA', 'vesselStatus', 'vesselDatasource', 'TRANSPORT_TRACE']
    
    train_df = pd.DataFrame()
    for chunk_num in range(14):
        print('\nfile{}'.format(chunk_num))
        chunk_data = pd.read_csv('../../dataset/train_wash/train_wash{}.csv'.format(chunk_num), header=None)
        chunk_data.columns = COLUMNS
        chunk_data = chunk_data.drop(['vesselNextport', 'vesselNextportETA', 'vesselStatus', 'vesselDatasource'], axis = 1)
        print('chunk_data\n{}'.format(len(chunk_data)))
        
        # 去除终点港口塞港数据
        chunk_data = drop_train_jam(chunk_data)
        chunk_data = get_data(chunk_data)
        
        # 经纬度编码
        a = 0.25  # 经纬度栅格化粒度
        chunk_data['geo_bin'] = int(360//a) * ((chunk_data['latitude'] + 90)//a).astype(int) + ((chunk_data['longitude'] + 360)//a).astype(int)
        chunk_data = chunk_data.drop_duplicates(['loadingOrder', 'geo_bin'], keep = 'last')
        print('geo_len:{}'.format(len(chunk_data)))
        
        group_df = chunk_data.groupby(['loadingOrder'], sort = False)['timestamp'].agg({'tmax': np.max}).reset_index()
        chunk_data = chunk_data.merge(group_df, on = 'loadingOrder', how = 'left')
        
        # 读取数据的最大值-最小值，即确认时间间隔为label
        chunk_data['label'] = (chunk_data['tmax'] - chunk_data['timestamp']).dt.total_seconds()/3600
    
        chunk_data = sample_order(chunk_data)
        print('match_len:{}'.format(len(chunk_data)))
        
        # 构建特征
        chunk_feature = get_feature(chunk_data, mode='train')
        
        train_df = train_df.append(chunk_feature, ignore_index = True)
        print('running file{}: {} seconds'.format(chunk_num, time.process_time() - start))
        
    train_df.to_csv('../../dataset/feature_gps/train_feature0809.csv', index = False)


def sample_order(df):
    match_train = pd.DataFrame()
    orders = df.loadingOrder.unique()
    for i in range(len(orders)):
        order_data = df[df['loadingOrder'] == orders[i]].reset_index(drop = True)
        trace = order_data.loc[0, 'TRANSPORT_TRACE']
        print('\norder{}/{}:{}\n{}'.format(i, len(orders), orders[i], trace))

        # 匹配测试集路由
        sample_trace = get_sample_trace(str(trace))  
        print('sample trace:{}'.format(sample_trace))
        
        match_test_data = test_data.loc[test_data['TRANSPORT_TRACE'].isin (sample_trace)]
        match_test_orders = match_test_data.loadingOrder.unique()  # 测试集匹配订单数量
        if (len(match_test_orders) > 0):
            # 遍历测试集中相同trace订单进行采样
            for match_test_order in match_test_orders:
                match_test_order_data = match_test_data[match_test_data['loadingOrder'] == match_test_order].reset_index(drop=True)
                    
                # 对每个训练集采样次数
                sample_len = len(match_test_order_data)  
                print('sample len: {}\torder_data: {}'.format(sample_len, (len(order_data))))
                
                # 保证测试集数据大于采样点
                if (sample_len > len(order_data)):
                    continue

                # 用测试集样本经纬度匹配路径经纬度
                start_lon = match_test_order_data.loc[0, 'longitude']
                start_lat = match_test_order_data.loc[0, 'latitude']
                end_lon = match_test_order_data.loc[len(match_test_order_data) - 1, 'longitude']
                end_lat = match_test_order_data.loc[len(match_test_order_data) - 1, 'latitude']

                # 距离匹配
                distance_to_start = []
                distance_to_end = []
                for i in range(len(order_data)):
                    distance1 = abs(order_data.loc[i, 'longitude'] - start_lon) + abs(order_data.loc[i, 'latitude'] - start_lat)
                    distance2 = abs(order_data.loc[i, 'longitude'] - end_lon) + abs(order_data.loc[i, 'latitude'] - end_lat)

                    distance_to_start.append(distance1)
                    distance_to_end.append(distance2)
                index_to_start = distance_to_start.index(min(distance_to_start))
                index_to_end = distance_to_end.index(min(distance_to_end))
                print('start:{}\tend:{}\tlen:{}'.format(index_to_start, index_to_end, index_to_end - index_to_start))
                
                if ((index_to_end - index_to_start) < 0):
                    continue

                if ((index_to_end - index_to_start) > sample_len):
                    sample_list = list(range(index_to_start, index_to_end))  # 大部分采样情况
                else:
                    sample_list = list(range(index_to_start, index_to_end + 1))
               
                    
                # 进行采样
                sample_list.sort()
                sample_data = order_data.iloc[sample_list, :].reset_index(drop=True)

                match_train = match_train.append(sample_data, ignore_index = True)
    match_train = match_train[order_data.columns]
    
    return match_train

def get_sample_trace(trace):
    sample_trace = []
    
    # 找相同路由
    trace_test = test_order.TRANSPORT_TRACE.unique()
    if trace in trace_test:
        sample_trace.append(trace)
        return sample_trace

    # 没有相同路由，找相近似路由
    trace_df = train_order[train_order['TRANSPORT_TRACE'] == trace].reset_index(drop = True)
    trace_num = trace_df.loc[0, 'trace']
    print(trace_num)
    
    match_test = test_order.loc[test_order['trace'] == trace_num].reset_index(drop = True)
    if (len(match_test) > 0):
        sample_trace.append(match_test.loc[0, 'TRANSPORT_TRACE'])
        return sample_trace
    
    return []

    
def get_test():
    test_raw = pd.read_csv('../../dataset/test_data/B_testData0807.csv')
    print(len(test_raw))
    
    # 测试集清洗
    test_raw = test_raw[test_raw['direction'] != -1]
    print(len(test_raw))
    
    # 清洗空值记录
    test_raw = test_raw.dropna(how = 'any', axis = 0)
    
    orders = test_raw.loadingOrder.unique()
    print('true order:{}'.format(len(orders)))
    
    test_data = pd.DataFrame()
    for num in range(len(orders)):
        order_data = test_raw[test_raw['loadingOrder'] == orders[num]].reset_index(drop=True)
        trace = str(order_data.loc[0, 'TRANSPORT_TRACE'])

        if (trace == 'BRSSZ-CNSHK'):
            order_data = order_data.loc[order_data['longitude'] > 117.8]

        order_data = order_data.drop_duplicates(['timestamp', 'vesselMMSI']).reset_index(drop=True)
        order_data['vesselMMSI'] = order_data.loc[0, 'vesselMMSI']
        
        test_data = test_data.append(order_data, ignore_index = True)
    test_data = test_data[order_data.columns]
    
    # 去除终点塞港数据
    test_data = drop_test_jam(test_data)
    test_data = get_data(test_data)
    
    # 经纬度编码
    a = 0.25  # 经纬度栅格化粒度
    test_data['geo_bin'] = int(360//a) * ((test_data['latitude'] + 90)//a).astype(int) + ((test_data['longitude'] + 360)//a).astype(int)
    test_data = test_data.drop_duplicates(['loadingOrder', 'geo_bin'], keep = 'last')
    print('geo_len:{}'.format(len(test_data)))
    
    # 记录每个订单最大时间戳
    group_df = test_data.groupby(['loadingOrder'], sort = False)['timestamp'].agg({'tmax': np.max}).reset_index()
    test_data = test_data.merge(group_df, on = 'loadingOrder', how = 'left')
        
    # 构建特征
    test_feature = get_feature(test_data, mode = 'test')
    
    print(len(test_feature.loadingOrder.unique()))
    test_feature.to_csv('../../dataset/feature_gps/test_feature0809.csv', index = False)
    print('running file: {} seconds'.format(time.process_time() - start))
    
    return test_data
    

    
def drop_train_jam(train_df):
    train_df = train_df.merge(train_order[['loadingOrder',  'start_port', 'end_port', 'start_lon', 'start_lat', 'end_lon', 'end_lat']],
                              on = 'loadingOrder', how = 'left')
    train_df['lon_diff_start'] = abs(train_df['start_lon'] - train_df['longitude'])
    train_df['lat_diff_start'] = abs(train_df['start_lat'] - train_df['latitude'])
        
    train_df['lon_diff_end'] = abs(train_df['end_lon'] - train_df['longitude'])
    train_df['lat_diff_end'] = abs(train_df['end_lat'] - train_df['latitude'])
    
    # 去除终点港口塞港数据
    train_df = train_df[(train_df['lon_diff_end'] > 0.25) | (train_df['lat_diff_end'] > 0.25)].reset_index(drop = True)
    train_df = train_df.drop(['lon_diff_start', 'lat_diff_start', 'lon_diff_end', 'lat_diff_end'], axis = 1)
    return train_df


def drop_test_jam(df):
    df = df.merge(test_order[['loadingOrder',  'start_port', 'end_port', 'start_lon', 'start_lat', 'end_lon', 'end_lat']],
                              on = 'loadingOrder', how = 'left')
    df['lon_diff_start'] = abs(df['start_lon'] - df['longitude'])
    df['lat_diff_start'] = abs(df['start_lat'] - df['latitude'])
        
    df['lon_diff_end'] = abs(df['end_lon'] - df['longitude'])
    df['lat_diff_end'] = abs(df['end_lat'] - df['latitude'])
    
    test_df = pd.DataFrame()
    orders = df.loadingOrder.unique()
    for i in range(len(orders)):
        # 去除终点港口塞港数据
        order_df = df[df['loadingOrder'] == orders[i]].reset_index(drop = True)
        order_df1 = order_df[(order_df['lon_diff_end'] > 0.25) | (order_df['lat_diff_end'] > 0.25)].reset_index(drop = True)
        
        # 测试集塞港，取第一条数据
        if (len(order_df1) == 0):
            order_df1 = order_df.iloc[0, :]
            
        test_df = test_df.append(order_df1, ignore_index = True)
    test_df = test_df[df.columns]
    test_df = test_df.drop(['lon_diff_start', 'lat_diff_start', 'lon_diff_end', 'lat_diff_end'], axis = 1)
    return test_df

    
def distance(LonA, LatA, LonB, LatB):
    EARTH_RADIUS = 6371  # 千米

    def rad(d):
        return d * np.pi/ 180.0

    radLatA = rad(LatA)
    radLatB = rad(LatB)

    a = radLatA-radLatB
    b = rad(LonA)-rad(LonB)

    s = 2 * np.arcsin(np.sqrt(np.power(np.sin(a / 2),2)+ np.cos(radLatA) * np.cos(radLatB)*np.power(np.sin(b / 2),2)))
    s = s* EARTH_RADIUS

    #  保留两位小数
    s = np.round(s * 100)/100
    return s

def get_data(data):
    data['timestamp'] = pd.to_datetime(data['timestamp'], infer_datetime_format=True)
    data['longitude'] = data['longitude'].astype(float)
    data['latitude'] = data['latitude'].astype(float)
    data['speed'] = data['speed'].astype(float)
    data['direction'] = data['direction'].astype(float) / 100

    return data

def is_covids(start_year, start_month):
    if (start_year == 2020) and (start_month >=2):
        return 1
    else:
        return 0

# 代码参考：https://github.com/juzstu/TianChi_HaiYang
def get_feature(df, mode = 'train'):
    # 经纬度编码
    a = 0.25  # 经纬度栅格化粒度
        
    # 速度、方向、经纬度离散化
    df['speed_bin'] =  df['speed']//5
    df['direction_bin'] =  df['direction']//5
    df['end_geo_bin'] =  int(360//a) * ((df['end_lat'] + 90)//a).astype(int) + ((df['end_lon'] + 360)//a).astype(int)


    # 空间信息
    df['lon_diff'] = abs(df['end_lon'] - df['start_lon'])
    df['lat_diff'] = abs(df['end_lat'] - df['start_lat'])
    df['lon_diff_bin'] = abs(df['end_lon'] - df['start_lon'])//0.5
    df['lat_diff_bin'] = abs(df['end_lat'] - df['start_lat'])//0.5

    df['lon_diff_start'] = abs(df['longitude'] - df['start_lon'])
    df['lat_diff_start'] = abs(df['latitude'] - df['start_lat'])
    df['lon_diff_start_bin'] = abs(df['longitude'] - df['start_lon'])//0.5
    df['lat_diff_start_bin'] = abs(df['latitude'] - df['start_lat'])//0.5
    df['dis_diff_start'] = df.apply(lambda x: distance(x['longitude'], x['latitude'], x['start_lon'], x['start_lat']), axis = 1)
    df['dis_diff_start_bin'] = df['dis_diff_start']//50

    df['lon_diff_end'] = abs(df['end_lon'] - df['longitude'])
    df['lat_diff_end'] = abs(df['end_lat'] - df['latitude'])
    df['lon_diff_end_bin'] = abs(df['end_lon'] - df['longitude'])//0.5
    df['lat_diff_end_bin'] = abs(df['end_lat'] - df['latitude'])//0.5
    df['dis_diff_end'] = df.apply(lambda x: distance(x['longitude'], x['latitude'], x['end_lon'], x['end_lat']), axis = 1)
    df['dis_diff_end_bin'] =  df['dis_diff_end']//50

    # 时间信息
    df['start_year'] = df['timestamp'].dt.year
    df['start_month'] = df['timestamp'].dt.month
    df['start_weekofyear'] = df['timestamp'].dt.weekofyear
    df['start_dayofweek'] = df['timestamp'].dt.dayofweek
    df['start_hour'] = df['timestamp'].dt.hour
    
    df['is_covids'] = df.apply(lambda x: is_covids(x['start_year'], x['start_month']), axis = 1)
    
    return df

if __name__ == "__main__":
    start = time.process_time()

    test_data = get_test()
    get_train()

    print('running time: {} seconds'.format(time.process_time() - start))

57071
56919
true order:239
geo_len:4939
239
running file: 9.289279030999978 seconds

file0
chunk_data
3056595
geo_len:165240

order0/437:AA191175561416
CNYTN-MXZLO
sample trace:['CNYTN-MXZLO']
sample len: 37	order_data: 608
start:0	end:60	len:60
sample len: 53	order_data: 608
start:0	end:66	len:66
sample len: 100	order_data: 608
start:0	end:271	len:271
sample len: 25	order_data: 608
start:0	end:58	len:58
sample len: 18	order_data: 608
start:0	end:21	len:21
sample len: 12	order_data: 608
start:0	end:22	len:22
sample len: 7	order_data: 608
start:2	end:22	len:20
sample len: 12	order_data: 608
start:0	end:33	len:33
sample len: 14	order_data: 608
start:1	end:21	len:20
sample len: 54	order_data: 608
start:4	end:79	len:75
sample len: 24	order_data: 608
start:0	end:60	len:60
sample len: 38	order_data: 608
start:0	end:61	len:61
sample len: 8	order_data: 608
start:0	end:7	len:7
sample len: 23	order_data: 608
start:6	end:59	len:53
sample len: 29	order_data: 608
start:0	end:61	len:61
sample len: 2

start:0	end:48	len:48
sample len: 43	order_data: 732
start:1	end:108	len:107
sample len: 11	order_data: 732
start:0	end:36	len:36
sample len: 29	order_data: 732
start:1	end:87	len:86
sample len: 36	order_data: 732
start:1	end:81	len:80
sample len: 33	order_data: 732
start:0	end:47	len:47
sample len: 21	order_data: 732
start:0	end:42	len:42
sample len: 28	order_data: 732
start:6	end:49	len:43

order25/437:AG424688892904
CNYTN-BRSSZ
sample trace:['CNYTN-BRSSZ']
sample len: 15	order_data: 835
start:62	end:74	len:12
sample len: 32	order_data: 835
start:95	end:123	len:28
sample len: 33	order_data: 835
start:259	end:291	len:32

order26/437:AG478264460880
CNYTN-BRSSZ
sample trace:['CNYTN-BRSSZ']
sample len: 15	order_data: 745
start:32	end:40	len:8
sample len: 32	order_data: 745
start:60	end:79	len:19
sample len: 33	order_data: 745
start:187	end:215	len:28

order27/437:AG781637728507
CNSHK-GRPIR-ITSPE-ITGOA
sample trace:['CNSHK-GRPIR-ITSPE-ITGOA']
sample len: 16	order_data: 616
start:449	end:4


order62/437:AP254924452907
CNSHK-CLVAP
sample trace:['CNSHK-CLVAP']
sample len: 41	order_data: 844
start:0	end:190	len:190
sample len: 86	order_data: 844
start:41	end:338	len:297
sample len: 90	order_data: 844
start:0	end:260	len:260
sample len: 2	order_data: 844
start:0	end:1	len:1
sample len: 7	order_data: 844
start:38	end:83	len:45
sample len: 2	order_data: 844
start:0	end:1	len:1
sample len: 7	order_data: 844
start:0	end:5	len:5
sample len: 31	order_data: 844
start:0	end:275	len:275
sample len: 60	order_data: 844
start:0	end:321	len:321

order63/437:AP321975865728
CNYTN-NLRTM
sample trace:['CNYTN-NLRTM']
sample len: 22	order_data: 502
start:9	end:32	len:23

order64/437:AP462750373009
CNSHK-BRSSZ
771-279
sample trace:['CNYTN-BRSSZ']
sample len: 15	order_data: 908
start:103	end:113	len:10
sample len: 32	order_data: 908
start:134	end:169	len:35
sample len: 33	order_data: 908
start:332	end:364	len:32

order65/437:AP915419941644
CNSHK-INNSA
sample trace:['CNSHK-INNSA']
sample len: 18	o

start:0	end:41	len:41
sample len: 28	order_data: 791
start:5	end:57	len:52

order84/437:AV581038137582
CNYTN-MATNG-MACAS
771-306
sample trace:['CNYTN-MYTPP-EGSUE-MATNG-MACAS']
sample len: 41	order_data: 477
start:405	end:437	len:32
sample len: 26	order_data: 477
start:61	end:74	len:13

order85/437:AV702266351407
CNSHK-INNSA
sample trace:['CNSHK-INNSA']
sample len: 18	order_data: 157
start:43	end:55	len:12
sample len: 2	order_data: 157
start:0	end:1	len:1

order86/437:AV717017640413
CNYTN-SGSIN-EGSUZ-GRPIR-ITSPE-ITGOA
sample trace:['CNYTN-SGSIN-EGSUZ-GRPIR-ITSPE-ITGOA']
sample len: 16	order_data: 591
start:445	end:457	len:12
sample len: 1	order_data: 591
start:577	end:577	len:0

order87/437:AW869931881979
CNSHK-MYPKG
sample trace:['CNSHK-MYPKG']
sample len: 5	order_data: 79
start:37	end:41	len:4
sample len: 13	order_data: 79
start:0	end:7	len:7
sample len: 6	order_data: 79
start:0	end:4	len:4
sample len: 7	order_data: 79
start:0	end:5	len:5
sample len: 28	order_data: 79
start:0	end:7	le

start:257	end:272	len:15

order112/437:BC348816687960
CNYTN-MYTPP-EGSUE-MATNG-MACAS
sample trace:['CNYTN-MYTPP-EGSUE-MATNG-MACAS']
sample len: 41	order_data: 380
start:327	end:348	len:21
sample len: 26	order_data: 380
start:74	end:85	len:11

order113/437:BC648986845768
CNYTN-PHBTG
sample trace:['CNYTN-PHBTG']
sample len: 7	order_data: 30
start:0	end:3	len:3
sample len: 3	order_data: 30
start:0	end:1	len:1
sample len: 3	order_data: 30
start:0	end:1	len:1
sample len: 10	order_data: 30
start:11	end:13	len:2

order114/437:BC855099532030
CNHKG-JPTYO
771-850
sample trace:['CNSHK-JPTYO']
sample len: 27	order_data: 114
start:0	end:24	len:24
sample len: 8	order_data: 114
start:0	end:6	len:6
sample len: 2	order_data: 114
start:0	end:1	len:1
sample len: 29	order_data: 114
start:22	end:45	len:23
sample len: 2	order_data: 114
start:0	end:1	len:1

order115/437:BC864042177995
CNSHK-PHBTG
771-811
sample trace:['CNYTN-PHBTG']
sample len: 7	order_data: 52
start:3	end:1	len:-2
sample len: 3	order_data: 5

...

start:371	end:407	len:36
sample len: 31	order_data: 498
start:76	end:93	len:17
sample len: 29	order_data: 498
start:257	end:272	len:15

order399/414:ZY272474172287
CNSHK-BRSSZ
771-279
sample trace:['CNYTN-BRSSZ']
sample len: 15	order_data: 765
start:31	end:43	len:12
sample len: 32	order_data: 765
start:62	end:81	len:19
sample len: 33	order_data: 765
start:209	end:238	len:29

order400/414:ZY281828841910
CNYTN-MYTPP-NLRTM
771-355
sample trace:['CNYTN-NLRTM']
sample len: 22	order_data: 713
start:39	end:80	len:41

order401/414:ZY294539775651
CNYTN-GBSOU
sample trace:['CNYTN-GBSOU']
sample len: 26	order_data: 744
start:314	end:345	len:31

order402/414:ZY663229186444
CNSHK-INNSA
sample trace:['CNSHK-INNSA']
sample len: 18	order_data: 180
start:40	end:52	len:12
sample len: 2	order_data: 180
start:0	end:1	len:1

order403/414:ZY669189594316
CNSHK-ZADUR
sample trace:['CNSHK-ZADUR']
sample len: 6	order_data: 360
start:1	end:16	len:15
sample len: 49	order_data: 360
start:71	end:311	len:240

order4

sample len: 1	order_data: 86
start:0	end:0	len:0
sample len: 1	order_data: 86
start:0	end:0	len:0
sample len: 6	order_data: 86
start:37	end:54	len:17
sample len: 15	order_data: 86
start:0	end:42	len:42
sample len: 2	order_data: 86
start:35	end:35	len:0
sample len: 21	order_data: 86
start:0	end:61	len:61

order4/59:AO393647206772X
HKHKG-FRFOS
sample trace:['HKHKG-FRFOS']
sample len: 43	order_data: 401
start:39	end:354	len:315

order5/59:AS316641737557X
CNSHK-GRPIR
sample trace:['CNSHK-GRPIR']
sample len: 47	order_data: 545
start:0	end:108	len:108

order6/59:BC159365740735X
CNSHK-MYTPP
sample trace:['CNSHK-MYTPP']
sample len: 9	order_data: 29
start:16	end:17	len:1
sample len: 13	order_data: 29
start:16	end:21	len:5
sample len: 9	order_data: 29
start:16	end:17	len:1
sample len: 7	order_data: 29
start:0	end:16	len:16
sample len: 2	order_data: 29
start:17	end:19	len:2
sample len: 1	order_data: 29
start:0	end:0	len:0
sample len: 2	order_data: 29
start:1	end:2	len:1
sample len: 3	order_data: 

start:50	end:359	len:309

order22/59:GQ733714774483X
CNSHK-MYTPP
sample trace:['CNSHK-MYTPP']
sample len: 9	order_data: 92
start:35	end:67	len:32
sample len: 13	order_data: 92
start:37	end:84	len:47
sample len: 9	order_data: 92
start:35	end:67	len:32
sample len: 7	order_data: 92
start:0	end:37	len:37
sample len: 2	order_data: 92
start:50	end:82	len:32
sample len: 1	order_data: 92
start:0	end:0	len:0
sample len: 2	order_data: 92
start:1	end:2	len:1
sample len: 3	order_data: 92
start:2	end:3	len:1
sample len: 2	order_data: 92
start:37	end:37	len:0
sample len: 1	order_data: 92
start:0	end:0	len:0
sample len: 1	order_data: 92
start:0	end:0	len:0
sample len: 6	order_data: 92
start:40	end:60	len:20
sample len: 15	order_data: 92
start:0	end:46	len:46
sample len: 2	order_data: 92
start:37	end:37	len:0
sample len: 21	order_data: 92
start:0	end:67	len:67

order23/59:GX940646127666X
CNSHK-MYTPP
sample trace:['CNSHK-MYTPP']
sample len: 9	order_data: 29
start:14	end:15	len:1
sample len: 13	order_da

sample len: 21	order_data: 92
start:0	end:67	len:67

order34/59:NI637318244875X
SGSIN-GRPIR
sample trace:['SGSIN-GRPIR']
sample len: 29	order_data: 426
start:50	end:76	len:26

order35/59:NZ438363836923X
HKHKG-FRFOS
sample trace:['HKHKG-FRFOS']
sample len: 43	order_data: 444
start:51	end:302	len:251

order36/59:OY743618853597X
CNSHK-MYTPP
sample trace:['CNSHK-MYTPP']
sample len: 9	order_data: 26
start:7	end:9	len:2
sample len: 13	order_data: 26
start:7	end:18	len:11
sample len: 9	order_data: 26
start:7	end:9	len:2
sample len: 7	order_data: 26
start:0	end:7	len:7
sample len: 2	order_data: 26
start:7	end:15	len:8
sample len: 1	order_data: 26
start:0	end:0	len:0
sample len: 2	order_data: 26
start:1	end:2	len:1
sample len: 3	order_data: 26
start:2	end:3	len:1
sample len: 2	order_data: 26
start:7	end:7	len:0
sample len: 1	order_data: 26
start:0	end:0	len:0
sample len: 1	order_data: 26
start:0	end:0	len:0
sample len: 6	order_data: 26
start:7	end:8	len:1
sample len: 15	order_data: 26
start:0	e

sample len: 3	order_data: 105
start:2	end:3	len:1
sample len: 2	order_data: 105
start:42	end:42	len:0
sample len: 1	order_data: 105
start:0	end:0	len:0
sample len: 1	order_data: 105
start:0	end:0	len:0
sample len: 6	order_data: 105
start:42	end:65	len:23
sample len: 15	order_data: 105
start:0	end:47	len:47
sample len: 2	order_data: 105
start:42	end:42	len:0
sample len: 21	order_data: 105
start:0	end:74	len:74

order50/59:ZI864782463424X
CNSHK-MYTPP
sample trace:['CNSHK-MYTPP']
sample len: 9	order_data: 42
start:14	end:23	len:9
sample len: 13	order_data: 42
start:14	end:34	len:20
sample len: 9	order_data: 42
start:14	end:23	len:9
sample len: 7	order_data: 42
start:0	end:14	len:14
sample len: 2	order_data: 42
start:18	end:32	len:14
sample len: 1	order_data: 42
start:0	end:0	len:0
sample len: 2	order_data: 42
start:1	end:2	len:1
sample len: 3	order_data: 42
start:2	end:3	len:1
sample len: 2	order_data: 42
start:14	end:14	len:0
sample len: 1	order_data: 42
start:0	end:0	len:0
sample len: 1